# Handling Missing Data

Why can a dataset have missing values? Sometimes an observation could not be recorded for whatever reason, like a sensor or instrument being broken or a survey respondent chose to not answer a question. Unfortunately, machine learning and statistical models often do not cope well with missing values. For this reason you will likely consider removing them. 

In this section, we will cover how to identify and remove data related to missing values.

> Please be advised you should fully understand why values are missing and trace back to the source of what produced the data. You should also be mindful of any selection biases that might emerge because of the missing data. For example, if you remove survey respondent records that did not answer a certain question... that can bias your models towards the population that chose to answer! Sometimes it is more interesting to ask why values are missing than just ignoring them. 

To get setup, let's bring in a SQLite database and extract the data from the `WEATHER_MONITOR` table. However, we will only analyze on region of stations by getting only records with `LOCATION_ID` of `2`,`28`, or `48`, and for the month of April. We will have SQL do that filtering. 

In [56]:
import urllib.request
import sqlite3
import pandas as pd 

# Download SQLite database and connect to it | link to database and then an orbitary name of the database
urllib.request.urlretrieve("https://github.com/thomasnield/machine-learning-demo-data/blob/master/relational/company_operations.db?raw=true", 'company_operations.db') 

conn = sqlite3.connect('company_operations.db') # Use that orbitary name here in the brackets

# FROM uses the name of the database (probably the name contained in the link, not the orbitrary name company_operations.db). 
# [strftime] is used to retrieve the 4th month in this case because this is a SQLite database.
# [parse_dates] means that to treat a column with date values as a date column
df = pd.read_sql("""
SELECT  * 
FROM    WEATHER_MONITOR 
WHERE   LOCATION_ID IN (2,28,48) 
        AND strftime('%m', REPORT_DATE) = '04'
""", conn, parse_dates=['REPORT_DATE'])
df

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.5,0,0.00,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.3,0,0.00,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.1,0,0.00,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.9,1,1.33,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.1,1,1.61,0.0,1,0,0
5,115,CYJP2T1,2021-04-01,48,56.4,1,NaN,0.0,0,0,0
6,132,RUJFBCR,2021-04-18,2,59.2,0,0.00,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.0,1,0.00,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.7,1,0.00,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.8,0,0.00,0.0,0,0,0


In [57]:
df.dtypes #with parsing

ID                      int64
REPORT_CODE            object
REPORT_DATE    datetime64[ns]
LOCATION_ID             int64
TEMPERATURE           float64
OVERCAST                int64
RAIN                  float64
SNOW                  float64
LIGHTNING               int64
HAIL                    int64
TORNADO                 int64
dtype: object

In [58]:
df.dtypes #without parsing

ID                      int64
REPORT_CODE            object
REPORT_DATE    datetime64[ns]
LOCATION_ID             int64
TEMPERATURE           float64
OVERCAST                int64
RAIN                  float64
SNOW                  float64
LIGHTNING               int64
HAIL                    int64
TORNADO                 int64
dtype: object

> If you want to learn more about SQL, check out the [Anaconda course here](https://learning.anaconda.cloud/introduction-to-sql).

Notice above that the `RAIN` column has `NaN` values, meaing those values are `None` and missing. There is also one record where a `TEMPERATURE` value is mssing. We are going to learn a few techniques on how to identify and handle these. 

## Tracking Missing Values

To find missing values, we can use the `isna()` function on a dataframe. 

In [59]:
df.isna()

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,True,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,False


We can also use the `any()` function to see which columns contain any missing values.

In [60]:
df.isna().any()

ID             False
REPORT_CODE    False
REPORT_DATE    False
LOCATION_ID    False
TEMPERATURE     True
OVERCAST       False
RAIN            True
SNOW           False
LIGHTNING      False
HAIL           False
TORNADO        False
dtype: bool

You can also flip the axis for `any()` and get a boolean series of whether each row contains a missing value. 

In [61]:
df.isna().any(axis=1)

0     False
1     False
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12     True
13    False
14    False
15    False
16    False
17    False
18     True
19     True
20    False
21    False
22    False
dtype: bool

In [62]:
df.notnull()

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,False,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,True


Note there is also a `notna()` counterpart which will flip the condition and set provided values to `True` and missing values to `False`. There are also aliases `isnull()` and `notnull()` which are just different names for the same operations.

Of course, we can take that boolean series and pass it to the `loc` getter to retrieve those columns with `NaN` values. 

In [63]:
df.loc[:, df.isna().any()]

,TEMPERATURE,RAIN
0,58.5,0.00
1,51.3,0.00
2,62.1,0.00
3,60.9,1.33
4,58.1,1.61
5,56.4,NaN
6,59.2,0.00
7,66.0,0.00
8,65.7,0.00
9,58.8,0.00


In [64]:
df.loc[df.isna().any(axis=1), :]

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
5,115,CYJP2T1,2021-04-01,48,56.4,1,NaN,0.0,0,0,0
12,466,U91W7VE,2021-04-15,28,NaN,0,0.0,0.0,0,0,0
18,713,ST9N6RZ,2021-04-29,28,56.0,1,NaN,0.0,0,1,1
19,716,87WYPTR,2021-04-30,48,61.0,1,NaN,0.0,0,1,1


To find missing values for specific columns, we can use a filtering operation using logical operators. Here we find all records where there was a missing `TEMPERATURE` or `RAIN` value.  

In [65]:
df[df['TEMPERATURE'].isna() | df['RAIN'].isna()]

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
5,115,CYJP2T1,2021-04-01,48,56.4,1,NaN,0.0,0,0,0
12,466,U91W7VE,2021-04-15,28,NaN,0,0.0,0.0,0,0,0
18,713,ST9N6RZ,2021-04-29,28,56.0,1,NaN,0.0,0,1,1
19,716,87WYPTR,2021-04-30,48,61.0,1,NaN,0.0,0,1,1


In [66]:
df[df['RAIN'].isna()]

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
5,115,CYJP2T1,2021-04-01,48,56.4,1,NaN,0.0,0,0,0
18,713,ST9N6RZ,2021-04-29,28,56.0,1,NaN,0.0,0,1,1
19,716,87WYPTR,2021-04-30,48,61.0,1,NaN,0.0,0,1,1


We can also filter for all records containing any missing values across all fields.

df[df.isna().any(axis=1)]

In [67]:
df[df.isna().any(axis=1)]

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
5,115,CYJP2T1,2021-04-01,48,56.4,1,NaN,0.0,0,0,0
12,466,U91W7VE,2021-04-15,28,NaN,0,0.0,0.0,0,0,0
18,713,ST9N6RZ,2021-04-29,28,56.0,1,NaN,0.0,0,1,1
19,716,87WYPTR,2021-04-30,48,61.0,1,NaN,0.0,0,1,1


## Removing Rows with Missing Values

As stated earlier, many machine learning and statistical models do not tolerate `NA`, `NaN`, or other missing null values. If you understand why they are missing, and do not think their absence will bias your model significantly, then you can simply remove those records with missing values. 

You can use the `drop()` operator with conditional logic as we learned in previous sections, but there is also a handy `dropna()` function just for this purpose. 

Below we use `dropna()` to remove all records with `NaN` values. Note I am not using the `inplace=True` parameter here so I can demonstrate other examples later. 

In [68]:
df.dropna(axis=0) # use inplace=True to replace current dataframe 

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.5,0,0.00,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.3,0,0.00,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.1,0,0.00,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.9,1,1.33,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.1,1,1.61,0.0,1,0,0
6,132,RUJFBCR,2021-04-18,2,59.2,0,0.00,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.0,1,0.00,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.7,1,0.00,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.8,0,0.00,0.0,0,0,0
10,318,EL7KNY6,2021-04-25,2,58.3,1,1.68,0.0,0,0,0


Notice those four records with `NaN` values for `RAIN` or `TEMPERATURE` are now gone. We can also provide only a `subset` of indices to consider for dropping null values. Below, we only drop records where `NA` exists in the `RAIN` column.  

In [72]:
df.dropna(axis=0, subset=["RAIN"])

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.5,0,0.00,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.3,0,0.00,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.1,0,0.00,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.9,1,1.33,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.1,1,1.61,0.0,1,0,0
6,132,RUJFBCR,2021-04-18,2,59.2,0,0.00,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.0,1,0.00,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.7,1,0.00,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.8,0,0.00,0.0,0,0,0
10,318,EL7KNY6,2021-04-25,2,58.3,1,1.68,0.0,0,0,0


If we would rather drop those columns with `NaN` values instead, we can use `axis=1`. 

In [73]:
df.dropna(axis=1)

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,OVERCAST,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,0,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,0,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,0,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,1,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,1,0.0,1,0,0
5,115,CYJP2T1,2021-04-01,48,1,0.0,0,0,0
6,132,RUJFBCR,2021-04-18,2,0,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,1,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,1,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,0,0.0,0,0,0


## Replacing Missing Values

While this may not make sense from a machine learning perspective, there may be times you want to replace missing values. You can do this by using the `fillna()` function. Below we replace all `na` values in our table with `-1`. Unfortunately, there is no `subset` parameter for this function so to target specific columns you will need to extract them out, apply the `fillna()` function, and then assign them back. 

In [74]:
df.fillna(value=-1, inplace=True)
df

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.5,0,0.00,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.3,0,0.00,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.1,0,0.00,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.9,1,1.33,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.1,1,1.61,0.0,1,0,0
5,115,CYJP2T1,2021-04-01,48,56.4,1,-1.00,0.0,0,0,0
6,132,RUJFBCR,2021-04-18,2,59.2,0,0.00,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.0,1,0.00,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.7,1,0.00,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.8,0,0.00,0.0,0,0,0


> There are other methods that `fillna()` can do to fill in missing values. [Be sure to read the Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna) to learn more. 

Conversely there may be times you want to replace certain values with `na`, such as empty strings or placeholder strings like 'NULL'. We just turned the `NaN` values into `-1`. Let's convert them back to `NaN` using the `replace()` function.

In [75]:
from numpy import nan

df.replace(-1, nan, inplace=True)
df

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.5,0,0.00,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.3,0,0.00,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.1,0,0.00,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.9,1,1.33,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.1,1,1.61,0.0,1,0,0
5,115,CYJP2T1,2021-04-01,48,56.4,1,NaN,0.0,0,0,0
6,132,RUJFBCR,2021-04-18,2,59.2,0,0.00,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.0,1,0.00,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.7,1,0.00,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.8,0,0.00,0.0,0,0,0


## Fill in Missing Values with Mean

Another way to cope with missing values that might be more agreeable for machine learning and statistical models is to use a statistical value replacement such as a `mean` or `median`. 

Let's bring in the `SimpleImputer` from scikit-learn and set it to use the `mean`. 

In [76]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

Let's then apply the `TEMPERATURE` and `RAIN` fields to the imputer. 

In [80]:
transform_input = df[['TEMPERATURE','RAIN']]

imputer.fit(transform_input)

SimpleImputer()

The `SimpleImputer` will calculate the mean for `RAIN` to be `0.8585` and for `TEMPERATURE` to be `59.740909`. We can then apply these columns with the means replacing the `NaN`'s to our dataframe.

In [83]:
# copy the weather dataframe 
mean_df = df.copy() 

# apply mean to the TEMPERATURE and RAIN column
transform_output = imputer.transform(transform_input)
mean_df[['TEMPERATURE','RAIN']] = transform_output
mean_df

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.500000,0,0.0000,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.300000,0,0.0000,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.100000,0,0.0000,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.900000,1,1.3300,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.100000,1,1.6100,0.0,1,0,0
5,115,CYJP2T1,2021-04-01,48,56.400000,1,0.8585,0.0,0,0,0
6,132,RUJFBCR,2021-04-18,2,59.200000,0,0.0000,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.000000,1,0.0000,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.700000,1,0.0000,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.800000,0,0.0000,0.0,0,0,0


Note there are other options for the `strategy` parameter including 'mean', 'median', 'most_frequent', and 'constant'.

## Fill in Missing Values with Nearest Neighbor

Another option for imputing a value to replace missing values is to leverage k-nearest neighbor (KNN), which works quite well in many cases. Essentially, the idea is to find datapoints that are close to the one with the missing value, all fields considered. Those neighboring records are then used to infer an estimate for the missing value. 

Let's bring in the `KNNInputer` and use the 5 nearest neighbors. We will make the weights uniform and we will tell it to not ignore `NaN` values by setting `metric` to `nan_euclidean`. 

In [84]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

In [85]:
df.dtypes

ID                      int64
REPORT_CODE            object
REPORT_DATE    datetime64[ns]
LOCATION_ID             int64
TEMPERATURE           float64
OVERCAST                int64
RAIN                  float64
SNOW                  float64
LIGHTNING               int64
HAIL                    int64
TORNADO                 int64
dtype: object

In [88]:
df2 = df['REPORT_DATE'].dt.strftime('%d')
df2

0     09
1     14
2     29
3     06
4     17
5     01
6     18
7     18
8     27
9     27
10    25
11    14
12    15
13    16
14    08
15    21
16    02
17    01
18    29
19    30
20    24
21    15
22    01
Name: REPORT_DATE, dtype: object

In [89]:
df3 = df2.astype(int)
df3.dtypes

dtype('int32')

Since we are inferring based on some other fields, we need to kick out `ID`,`LOCATION_ID`, and `REPORT_CODE` as those are not useful for the KNN model. They are arbitrary values or randomly generated and have no predictive value. We also need to convert the `REPORT_DATE` to a numeric value. Thankfully we are only working with one month, so let's just grab the day of month and that will be our numeric conversion.

In [90]:
# copy the datafarme and drop two columns that are not useful for modeling
knn_input = df.drop(['ID','REPORT_CODE','LOCATION_ID'],axis=1)

# extract day of month and make that the `REPORT_DATE`
knn_input['REPORT_DATE'] = knn_input['REPORT_DATE'].dt.strftime('%d').astype(int)

# fit the knn model 
imputer.fit(knn_input)
knn_input

,REPORT_DATE,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,9,58.5,0,0.00,0.0,0,0,0
1,14,51.3,0,0.00,0.0,0,0,0
2,29,62.1,0,0.00,0.0,0,0,0
3,6,60.9,1,1.33,0.0,1,0,0
4,17,58.1,1,1.61,0.0,1,0,0
5,1,56.4,1,NaN,0.0,0,0,0
6,18,59.2,0,0.00,0.0,0,0,0
7,18,66.0,1,0.00,0.0,0,1,1
8,27,65.7,1,0.00,0.0,1,0,0
9,27,58.8,0,0.00,0.0,0,0,0


Finally, let's copy the dataframe and apply the transformation to the two columns. The `RAIN` values with row indices of 5, 18, and 19 were missing but now are `0.604`, `0.996`, and `0.986` respectively. The missing `TEMPERATURE` value of row index 12 is inferred to be `55.76`. 

In [91]:
# copy the dataframe
knn_output = df.copy()

# apply knn transform to the input
knn_transform = imputer.transform(knn_input)

# apply only the TEMPERATURE and RAIN columns back to the dataframe
knn_output.loc[:,["TEMPERATURE","RAIN"]] = knn_transform[:,[1,3]]
knn_output

,ID,REPORT_CODE,REPORT_DATE,LOCATION_ID,TEMPERATURE,OVERCAST,RAIN,SNOW,LIGHTNING,HAIL,TORNADO
0,5,EHVUPGY,2021-04-09,48,58.50,0,0.000,0.0,0,0,0
1,13,NJOJ7KK,2021-04-14,2,51.30,0,0.000,0.0,0,0,0
2,49,N9LZ2TF,2021-04-29,48,62.10,0,0.000,0.0,0,0,0
3,56,JWSTJ8A,2021-04-06,2,60.90,1,1.330,0.0,1,0,0
4,99,93E6EOR,2021-04-17,28,58.10,1,1.610,0.0,1,0,0
5,115,CYJP2T1,2021-04-01,48,56.40,1,0.604,0.0,0,0,0
6,132,RUJFBCR,2021-04-18,2,59.20,0,0.000,0.0,0,0,0
7,187,TV9XROC,2021-04-18,48,66.00,1,0.000,0.0,0,1,1
8,205,A66GZWC,2021-04-27,48,65.70,1,0.000,0.0,1,0,0
9,242,4BP1L1R,2021-04-27,28,58.80,0,0.000,0.0,0,0,0


## Exercise

In the code below is a sample of thermostat data. Complete the code below so the median is imputed for the `temperature` and `humidity` fields' missing values. 

In [9]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, None, 57.2, 57.4, None, 27.5], 
    "humidity" : [.8, None, .7, .6, .7, .7, .8]
})

# create and fit imputer 
imputer = SimpleImputer(strategy='median')
transform_input = df[['temperature','humidity']]
imputer.fit(transform_input)

# apply mean to the temperature and humidity column
transform_output = imputer.transform(transform_input)
df[['temperature','humidity']] = transform_output
df

,record_id,temperature,humidity
0,OVUTJE,65.2,0.8
1,OVUTJE,65.2,0.7
2,WI4QEX,57.4,0.7
3,WI4QEX,57.2,0.6
4,FS40NF,57.4,0.7
5,O64LIT,57.4,0.7
6,U888EA,27.5,0.8


In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, None, 57.2, 57.4, None, 27.5], 
    "humidity" : [.8, None, .7, .6, .7, .7, .8]
})

# create and fit imputer 
imputer = SimpleImputer(strategy='median')
transform_input = df[['temperature','humidity']]
imputer.fit(transform_input)

# apply mean to the TEMPERATURE and RAIN column
transform_output = imputer.transform(transform_input)
df[['temperature','humidity']] = transform_output
df